# Experiments with models from RecTools

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Set a random seed for reproducibility
np.random.seed(42)

# Generate a DataFrame with 100 rows
data = []
start_time = pd.Timestamp.now()
time_step = timedelta(minutes=1)

for i in range(100):
    array1 = np.random.randint(-10, 10, size=(2,))
    array2 = np.random.randint(-10, 10, size=(2,))
    label = np.random.choice([-1, 1], size=(1,))[0]
    timestamp = start_time + i * time_step
    data.append([i, i+100, label, timestamp, array1.tolist(), array2.tolist()])
    # data.append([i, i+100, label, timestamp])
    # data.append([array1.tolist(), array2.tolist(), label, timestamp])

# Create the DataFrame
columns = ['User', 'Item', 'weight', 'Timestamp', 'Array1', 'Array2']
df = pd.DataFrame(data, columns=columns)
df.to_csv('../data/temp.csv')
# Display the DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   User       100 non-null    int64         
 1   Item       100 non-null    int64         
 2   weight     100 non-null    int64         
 3   Timestamp  100 non-null    datetime64[ns]
 4   Array1     100 non-null    object        
 5   Array2     100 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.8+ KB
None


## Reading data into RecTools format

In [2]:
from rectools import Columns
from rectools.dataset import Dataset, DenseFeatures
import pandas as pd
import pickle


quotes_labeled_reddit = pd.read_csv('../data/quotes_labeled_reddit.csv', index_col=0)
quotes_labeled_reddit = quotes_labeled_reddit.replace('\r\n','\n', regex=True)
# quotes_reddit_dense = DenseFeatures.from_iterables(
#     [el.to_list() for i, el in quotes_labeled_reddit.drop('Text', axis=1).iterrows()],
#     names=quotes_labeled_reddit.columns.to_list()[1:]
#     )
# print(quotes_reddit_dense)
diaries_labeled_reddit = pd.read_csv('../data/diaries_labeled_reddit.csv', index_col=0)
# diaries_reddit_dense = DenseFeatures.from_iterables(
#     [el.to_list() for i, el in diaries_labeled_reddit.drop('Text', axis=1).iterrows()],
#     names=diaries_labeled_reddit.columns.to_list()[1:]
#     )
# print(diaries_labeled_reddit)

In [10]:
from datetime import timedelta


rectools_data = []
rectools_diary_dense_features = []
rectools_quote_dense_features = []
rectools_diary_dense_fnames = None
rectools_quote_dense_fnames = None
diaries_index = set()
quotes_index = set()
start_time = pd.Timestamp.now()
time_step = timedelta(minutes=1)
for i, row in raw_interactions.iterrows():
    quote = quotes_labeled_reddit[quotes_labeled_reddit['Text'] == row['Quote']]
    quote_index = int(quote.index[0])
    quote_features = quote.drop('Text', axis=1).iloc[0].to_list()

    diary = diaries_labeled_reddit[diaries_labeled_reddit['Text'] == row['Text']]
    diary_index = int(diary.index[0])
    diary_features = diary.drop('Text', axis=1).iloc[0].to_list()

    timestamp = start_time + i * time_step
    if rectools_diary_dense_fnames is None:
        quote_fnames = quote.columns.to_list()[1:]
        diary_fnames = diary.columns.to_list()[1:]
        rectools_diary_dense_fnames = ['id'] + diary_fnames
        rectools_quote_dense_fnames = ['id'] + quote_fnames
    rectools_data.append([diary_index, quote_index, row['Interaction'], timestamp])
    if diary_index not in diaries_index:
        diaries_index.add(diary_index)
        rectools_diary_dense_features.append([diary_index] + diary_features)
    if quote_index not in quotes_index:
        quotes_index.add(quote_index)
        rectools_quote_dense_features.append([quote_index] + quote_features)

rectools_df = pd.DataFrame(rectools_data, columns=[*Columns.Interactions])
user_features_df = pd.DataFrame(rectools_diary_dense_features, columns=rectools_diary_dense_fnames)
item_features_df = pd.DataFrame(rectools_quote_dense_features, columns=rectools_quote_dense_fnames)
rectools_ds = Dataset.construct(rectools_df,
                            user_features_df=user_features_df,
                            cat_user_features=rectools_diary_dense_fnames,
                            item_features_df=item_features_df,
                            cat_item_features=rectools_quote_dense_fnames,
                            make_dense_user_features=True,
                            make_dense_item_features=True,
                            )
rectools_ds

Dataset(user_id_map=IdMap(external_ids=array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172

In [11]:
from rectools.models import DSSMModel


model = DSSMModel(rectools_ds, 
                  max_epochs = 10,
                  batch_size = 64
                 )
model.fit(rectools_ds)

[2023-11-28 11:11:30,485] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


AttributeError: 'Dataset' object has no attribute 'from_dataset'